# 03 Decoding Examples

This notebook walks through some simple examples for decoding the data.

### 0. Data Loading

In [ ]:
# Setting working directory to root to mimic usual usecase
###############################################################
import sys
import os
root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, root_path)
###############################################################
import numpy as np
import pandas as pd
###############################################################

from tutorial.utils import data_tools

In [ ]:
data_path = '../../data_test'
dates = data_tools.extract_dates_from_filenames(data_path)
data_CO, data_RD = data_tools.load_day(dates[0], data_path)

### 1. Ridge Regression Example

This example loads one day and perform ridge regression to predict finger kinematics. We start by splitting and augementing the data.

In [ ]:
# Load sbp, finger kinematics, and trial index data from day 42
date = dates[42]
data_CO, data_RD = data_tools.load_day(date, data_path)
if data_CO and data_RD:
    TrialIndex = max(data_CO['trial_index'], data_RD['trial_index'])
    sbp = np.concatenate((data_CO['sbp'], data_RD['sbp']), axis=0)
    beh = np.concatenate((data_CO['finger_kinematics'], data_RD['finger_kinematics']), axis=0)
elif data_RD:
    TrialIndex = data_RD['trial_index']
    sbp = data_RD['sbp']
    beh = data_RD['finger_kinematics']
else:
    TrialIndex = data_CO['trial_index']
    sbp = data_CO['sbp']
    beh = data_CO['finger_kinematics']

# Train-test split
test_len = np.min((len(TrialIndex)-1, 399))
neural_train= sbp[:TrialIndex[300]]
neural_test = sbp[TrialIndex[300]:TrialIndex[test_len]]
finger_train = beh[:TrialIndex[300]]
finger_test = beh[TrialIndex[300]:TrialIndex[test_len]]

# Add history
def add_hist(X, Y, hist=10):
    nNeu = X.shape[1]

    adjX = np.zeros((X.shape[0]-hist, nNeu, hist+1))
    for h in range(hist+1):
        adjX[:,:,h] = X[h:X.shape[0]-hist+h,:]
    adjY = Y[hist:,:]

    adjX = adjX.reshape(adjX.shape[0],-1)
    return adjX, adjY

neural_train, finger_train = add_hist(neural_train, finger_train, hist = 20)
neural_test, finger_test = add_hist(neural_test, finger_test, hist = 20)

# Add bias term
neural_test = np.concatenate((neural_test, np.ones((len(neural_test), 1))), axis=1)
neural_train = np.concatenate((neural_train, np.ones((len(neural_train), 1))), axis=1)

Then ridge regression is applied to the data and performance is evaluated using Pearson r.

In [ ]:
from sklearn import linear_model
from scipy.stats import pearsonr

reg = linear_model.Ridge(alpha = 0.001,fit_intercept=False)
reg.fit(neural_train, finger_train)
rr_prediction = reg.predict(neural_test)
r = np.array([pearsonr(finger_test, rr_prediction).statistic]).reshape(1,4)
print("Pearson r for each output dimension:", r)